In [ ]:
import numpy as np
import scipy.optimize as opt
import matplotlib.pyplot as plt

def amdahl(N, P):
    """Amdahl's Law function."""
    return 1 / ((1 - P) + P / N)

def estimate_parallel_fraction(cores, execution_times):
    """Estimate the parallel fraction P using Amdahl's Law."""
    # Compute speedup
    speedup = execution_times[0] / execution_times
    
    # Define error function for optimization
    def error_func(P):
        return np.sum((amdahl(cores, P) - speedup) ** 2)
    
    # Optimize for P
    P_opt = opt.minimize_scalar(error_func, bounds=(0, 1), method='bounded').x
    
    # Compute estimated speedup using the fitted P
    S_estimated = amdahl(cores, P_opt)
    
    return P_opt, S_estimated, speedup

def plot_results(cores, speedup, S_estimated):
    """Plot actual vs estimated speedup."""
    plt.figure(figsize=(8, 6))
    plt.plot(cores, speedup, marker='s', linestyle='-', color='#102C53', label='Measured Speedup')
    plt.plot(cores[0:7], cores[0:7], linestyle='--', color='#73A2D1', label='Ideal Speedup')  # Ideal linear speedup
    
    # Compute theoretical speedup curve
    theoretical_speedup_curve = np.array([amdahl(N, P_opt) for N in cores])
    plt.plot(cores, theoretical_speedup_curve, linestyle='-.', color='#73A2D1', label='Interpolated Amdahl Speedup')
    
    plt.xlabel('Number of Cores')
    plt.ylabel('Speedup')
    plt.title("Strong Scalability Test: Amdahl's Law Fit")
    plt.legend()
    plt.grid()
    plt.show()

if __name__ == "__main__":
    # Given data
    cores = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14])
    execution_times = np.array([
        4.52154, 2.05317, 1.47584, 1.03565, 1.00601, 0.718886,
        0.796677, 0.839686, 0.78654, 0.785686, 0.721863, 0.733553,
        0.821222, 0.712032
    ]) 
    """execution_times = np.array([
        7.18702, 4.75227, 3.46536, 2.64444, 2.24207, 1.94719, 1.76675, 1.50614, 1.68177, 1.60631, 1.83596, 1.69754, 1.89851, 1.84226
    ]) """
    
    # Estimate parallel fraction and speedup
    P_opt, S_estimated, speedup = estimate_parallel_fraction(cores, execution_times)
    
    # Display results
    print(f"Estimated Parallel Fraction (P): {P_opt:.4f}")
    print("Estimated Speedup:")
    for c, s in zip(cores, S_estimated):
        print(f"Cores: {c}, Speedup: {s:.4f}")
    
    # Plot results
    plot_results(cores, speedup, S_estimated)


In [ ]:
import numpy as np
import scipy.optimize as opt
import matplotlib.pyplot as plt

def gustafson_law(cores, s):
    """Function based on Gustafson's Law to calculate speedup, with p = 1 - s."""
    p = 1 - s  # Since s + p = 1
    return s + p * cores

def estimate_parallel_fraction_and_serial_fraction(cores, speedup):
    """Estimate the serial fraction (s) from the data using Gustafson's Law, with s + p = 1."""
    
    # Define the error function for optimization
    def error_func(s):
        p = 1 - s  # Ensure s + p = 1
        # Calculate the predicted speedup based on Gustafson's Law
        predicted_speedup = gustafson_law(cores, s)
        # Calculate the squared error between actual speedup and predicted speedup
        return np.sum((predicted_speedup - speedup) ** 2)
    
    # Use optimization to find the best s
    result = opt.minimize_scalar(error_func, bounds=(0, 1), method='bounded')
    
    # Extract optimal s
    s_opt = result.x
    p_opt = 1 - s_opt  # Calculate p from s

    # Compute the estimated speedup using the fitted parameters
    estimated_speedup = gustafson_law(cores, s_opt)
    
    return s_opt, p_opt, estimated_speedup

def plot_results(cores, speedup, s_opt, p_opt):
    """Plot actual vs estimated speedup with Gustafson's Law and fitted parameters."""
    plt.figure(figsize=(8, 6))
    
    # Measured Speedup
    plt.plot(cores, speedup, marker='s', linestyle='-', color='#102C53', label='Measured Speedup')

    # Ideal Linear Speedup
    plt.plot(cores, cores, linestyle='--', color='#73A2D1', label='Ideal Speedup')

    # Gustafson's Law Speedup (using fitted s and p)
    gustafson_speedup = gustafson_law(cores, s_opt)
    plt.plot(cores, gustafson_speedup, linestyle='-.', color='#73A2D1', label="Fitted Gustafson's Law")

    # Labels and legend
    plt.xlabel('Number of Cores')
    plt.ylabel('Speedup')
    plt.title("Weak Scalability Test: Gustafson's Law Fit")
    plt.legend()
    plt.grid()
    plt.show()

if __name__ == "__main__":
    # Given data
    cores = np.array([1, 2, 4, 8])
    speedup = np.array([1, 1.56, 3.12, 6.0069])

    # Estimate serial fraction (s) and parallel fraction (p)
    s_opt, p_opt, estimated_speedup = estimate_parallel_fraction_and_serial_fraction(cores, speedup)

    # Plot results with the estimated parameters
    plot_results(cores, speedup, s_opt, p_opt)

    # Print the estimated values of s and p
    print(f"Estimated Serial Fraction (s): {s_opt}")
    print(f"Estimated Parallel Fraction (p): {p_opt}")

